In [1]:
import pandas as pd
import numpy as np
from math import sqrt
import pytz
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt

In [2]:
df_el = pd.read_csv('./observation/East Lansing 5 min_crc.csv')
df_gl = pd.read_csv('./observation/CRNH0203-2007-2018-MI_Gaylord_9_SSW_crc.csv')
df_gr = pd.read_csv('./observation/ASOS Grand Rapids_crc.csv')
df_ele = pd.read_csv('./observation/East Leland 5 min.csv')
df_sh = pd.read_csv('./observation/South Haven 5 min_crc.csv')
df_tvc = pd.read_csv('./observation/ASOS Traverse City_crc_1.csv')

df_ele = df_ele.rename(columns={'year':'YEAR', 'month':'MONTH', 'day':'DAY', 'hour':'HOUR'})
df_sh = df_sh.rename(columns={'year':'YEAR', 'month':'MONTH', 'day':'DAY', 'hour':'HOUR', 'relh%':'relh', 'atmpF':'atmp', 'pcpn(in)':'pcpn'})
df_tvc['pcpn'] = np.nan

df_obs = [df_el, df_gl, df_gr, df_ele, df_sh, df_tvc]

In [35]:
atmp = []
relh = []
for i in range(6):
    atmp.append(np.array(df_obs[i].groupby([df_obs[i].YEAR,df_obs[i].MONTH,df_obs[i].DAY])['atmp'].mean()))
    relh.append(np.array(df_obs[i].groupby([df_obs[i].YEAR,df_obs[i].MONTH,df_obs[i].DAY])['relh'].mean()))

In [53]:
import pytz
eastern = pytz.timezone('US/Eastern')
dff = [pd.date_range(start='1/1/2001 0:00', end='12/31/2018 18:00', freq='H')[1:].tz_localize(pytz.utc).tz_convert(eastern),\
        pd.date_range(start='9/19/2007 17:00', end='12/31/2018 19:00', freq='H')[1:].tz_localize(pytz.utc).tz_convert(eastern),\
        pd.date_range(start='1/1/2001 00:00', end='12/30/2018 23:00', freq='H')[1:].tz_localize(pytz.utc).tz_convert(eastern),\
        pd.date_range(start='5/2/2003 00:00', end='1/1/2019 00:00', freq='H')[1:].tz_localize(pytz.utc).tz_convert(eastern),\
        pd.date_range(start='4/6/2006 00:00', end='1/1/2019 00:00', freq='H')[1:].tz_localize(pytz.utc).tz_convert(eastern),\
        pd.date_range(start='1/1/2001 00:00', end='12/30/2018 23:00', freq='H')[1:].tz_localize(pytz.utc).tz_convert(eastern)]

dff_obs = []
for i in range(6):
    year = dff[i].strftime('%Y')
    mon = dff[i].strftime('%-m')
    day = dff[i].strftime('%-d')
    dff_obs.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'atmp':df_obs[i]['atmp'], 'relh':df_obs[i]['relh'], 
                                'prcp':df_obs[i]['pcpn']}))

### Calculating missing percent

In [54]:
missing = []
for i in range(6):
    missing.append(((dff_obs[i].isnull() | dff_obs[i].isna()).sum() * 100 / dff_obs[i].index.size).round(2))

In [61]:
locn = ['East Lansing', 'Gaylord', 'Grand Rapids', 'East Leland', 'South Haven', 'Traverse City']
df_missing = pd.DataFrame({'Station':locn, 'atmp_missing%':[missing[i].atmp for i in range(6)], 
                           'relh_missing%':[missing[i].relh for i in range(6)],
                          'prcp_missing%':[missing[i].prcp for i in range(6)]})
df_missing

,Station,atmp_missing%,relh_missing%,prcp_missing%
0,East Lansing,1.38,0.00,0.13
1,Gaylord,0.06,26.63,0.69
2,Grand Rapids,1.97,2.39,54.91
3,East Leland,0.37,0.37,0.96
4,South Haven,0.94,0.99,0.80
5,Traverse City,3.06,3.88,100.00


In [5]:
# df_gl = pd.read_table('./observation/CRNH0203-2007-2018-MI_Gaylord_9_SSW.txt', delim_whitespace=True, 
#                                    names=('UTC_DATE','UTC_TIME','LST_DATE','LST_TIME','CRX_VN','LONGITUDE','LATITUDE','T_CALC',
#                                           'T_HR_AVG','T_MAX','T_MIN','P_CALC','SOLARAD','SOLARAD_FLAG','SOLARAD_MAX','SOLARAD_MAX_FLAG',
#                                           'SOLARAD_MIN', 'SOLARAD_MIN_FLAG','SUR_TEMP_TYPE','SUR_TEMP','SUR_TEMP_FLAG','SUR_TEMP_MAX',
#                                           'SUR_TEMP_MAX_FLAG','SUR_TEMP_MIN','SUR_TEMP_MIN_FLAG','RH_HR_AVG','RH_HR_AVG_FLAG',
#                                           'SOIL_MOISTURE_5','SOIL_MOISTURE_10','SOIL_MOISTURE_20','SOIL_MOISTURE_50','SOIL_MOISTURE_100',
#                                           'SOIL_TEMP_5','SOIL_TEMP_10','SOIL_TEMP_20','SOIL_TEMP_50','SOIL_TEMP_100'))
# rh = np.where(df_gl.RH_HR_AVG == -9999, np.nan, df_gl.RH_HR_AVG)
# sum(np.isnan(rh)) / len(rh)

In [2]:
df_el = pd.read_csv('./observation/East Lansing 5 min_crc.csv')
df_gl = pd.read_csv('./observation/CRNH0203-2007-2018-MI_Gaylord_9_SSW_crc.csv')
df_gr = pd.read_csv('./observation/ASOS Grand Rapids_crc.csv')
era_el = pd.read_csv('./extracted/East_lansing/East_Lansing_9km.csv')
era_gl = pd.read_csv('./extracted/Gaylord/Gaylord_9km.csv')
era_gr = pd.read_csv('./extracted/GRR/GRR_9km.csv')

df_gl = df_gl.drop(df_gl.index[-1])
era_gl = era_gl.drop(era_gl.index[:-98905])
df_gl = df_gl.reset_index(drop=True)
era_gl = era_gl.reset_index(drop=True)
era_gr = era_gr.drop(era_gr.index[157751:])

df_ele = pd.read_csv('./observation/East Leland 5 min.csv')
df_sh = pd.read_csv('./observation/South Haven 5 min_crc.csv')
era_ele = pd.read_csv('./extracted/East_Leland/East_Leland_9km.csv')
era_sh = pd.read_csv('./extracted/South_Haven/South_Haven_9km.csv')
df_ele = df_ele[:-6]
era_ele = era_ele[20424:]
df_sh = df_sh[:-6]
era_sh = era_sh[46104:]
df_tvc = pd.read_csv('observation/ASOS Traverse City_crc_1.csv')
era_tvc = pd.read_csv('extracted/TVC/TVC_9km.csv')[:-19]
df_ele = df_ele.reset_index(drop=True)
df_sh = df_sh.reset_index(drop=True)
df_tvc = df_tvc.reset_index(drop=True)
era_ele = era_ele.reset_index(drop=True)
era_sh = era_sh.reset_index(drop=True)
era_tvc = era_tvc.reset_index(drop=True)

df_ele = df_ele.rename(columns={'year':'YEAR', 'month':'MONTH', 'day':'DAY', 'hour':'HOUR'})
df_sh = df_sh.rename(columns={'year':'YEAR', 'month':'MONTH', 'day':'DAY', 'hour':'HOUR', 'relh%':'relh', 'atmpF':'atmp', 'pcpn(in)':'pcpn'})
df_tvc['pcpn'] = np.nan

obs = [df_el, df_ele, df_sh, df_gr, df_tvc, df_gl]
era = [era_el, era_ele, era_sh, era_gr, era_tvc, era_gl]

In [3]:
def rmse(df1,df2):
    return sqrt(mse(df1.loc[~df1.isnull()], df2.loc[~df1.isnull()]))

### Hourly ERA5 and Obervation RMSE

In [33]:
rmse_t = [rmse((obs[i].atmp-32)*5./9., (era[i].atmp-32)*5./9.) for i in range(6) if obs[i].atmp.isna().sum() < len(obs[i].atmp)]
rmse_rh = [rmse(obs[i].relh, era[i].relh) for i in range(6) if obs[i].relh.isna().sum() < len(obs[i].relh)]
rmse_prcp_5 = [rmse(obs[i].pcpn*25.4, era[i].pcpn*25.4) for i in range(6) if obs[i].pcpn.isna().sum() < len(obs[i].pcpn)]
rmse_prcp = rmse_prcp_5[:5]
rmse_prcp.extend([np.nan, rmse_prcp_5[-1]])

In [34]:
locn = ['East Lansing', 'East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord']
df_rmse = pd.DataFrame({'Station':[locn[i] for i in range(6)],
                        'Number':[len(obs[i]) for i in range(6)],
                        'Temperature RMSE (C)':[round(rmse_t[i],2) for i in range(6)],
                        'Relative Humidity RMSE (%)':[round(rmse_rh[i],2) for i in range(6)],
                        'Precipitation RMSE (mm)':[round(rmse_prcp[i],2) for i in range(6)],
                        'Temperature Bias (C)':[round(np.nanmean((era[i].atmp - obs[i].atmp)*5./9.), 2) for i in range(6)],
                        'Relative Humidity Bias (%)':[round(np.nanmean(era[i].relh - obs[i].relh), 2) for i in range(6)],
                        'Precipitation Bias (mm)':[round(np.nanmean((era[i].pcpn - obs[i].pcpn)*25.4), 2) for i in range(6)]
                       })
df_rmse.loc['median'] = df_rmse.median()
df_rmse

/home/mmfire/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':


,Station,Number,Temperature RMSE (C),Relative Humidity RMSE (%),Precipitation RMSE (mm),Temperature Bias (C),Relative Humidity Bias (%),Precipitation Bias (mm)
0,East Lansing,157770.0,1.810,11.260,0.43,0.37,-3.75,0.10
1,East Leland,137346.0,3.050,11.850,0.40,0.27,0.33,0.11
2,South Haven,111666.0,3.180,12.970,0.51,0.30,3.45,0.12
3,Grand Rapids,157751.0,1.570,9.060,1.02,-0.32,-0.09,-0.03
4,Traverse City,157751.0,2.160,9.590,0.60,-0.69,2.76,NaN
5,Gaylord,98905.0,1.710,9.590,NaN,0.23,-0.15,0.00
median,NaN,147548.5,1.985,10.425,0.51,0.25,0.12,0.10


### Daily statistics

In [66]:
# obs_tmin = [obs[i].groupby([obs[i].YEAR,obs[i].MONTH,obs[i].DAY])['atmp'].min() for i in range(6)]
# obs_tmax = [obs[i].groupby([obs[i].YEAR,obs[i].MONTH,obs[i].DAY])['atmp'].max() for i in range(6)]

# era_tmin = [era[i].groupby([era[i].YEAR,era[i].MONTH,era[i].DAY])['atmp'].min() for i in range(6)]
# era_tmax = [era[i].groupby([era[i].YEAR,era[i].MONTH,era[i].DAY])['atmp'].max() for i in range(6)]

In [5]:
# rmse_eratmin = [rmse((obs_tmin[i]-32)*5./9., (era_tmin[i]-32)*5./9.) for i in range(6)]
# rmse_eratmax = [rmse((obs_tmax[i]-32)*5./9., (era_tmax[i]-32)*5./9.) for i in range(6)]

In [4]:
# locn = ['East Lansing', 'East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord']
# df_rmse = pd.DataFrame({'Station':[locn[i] for i in range(6)],
#                         'Number':[len(obs[i]) for i in range(6)],
#                         'DminT RMSE (C)':[round(rmse_eratmin[i],2) for i in range(6)],
#                         'DmaxT RMSE (C)':[round(rmse_eratmax[i],2) for i in range(6)],
#                         'DminT Bias (C)':[round(np.nanmean((era_tmin[i]-obs_tmin[i])*5./9.),2) for i in range(6)],
#                         'DmaxT Bias (C)':[round(np.nanmean((era_tmax[i]-obs_tmax[i])*5./9.),2) for i in range(6)]})
# df_rmse.loc['median'] = df_rmse.median()
# df_rmse

### PRISM DAY is 24 hours from 7:00 a.m.

In [4]:
dff = [pd.date_range(start='1/1/2001 6:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='5/2/2003 6:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='4/6/2006 6:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='1/1/2001 6:00', end='12/31/2018 4:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='1/1/2001 6:00', end='12/31/2018 4:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='9/19/2007 23:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc)]
obs_utc = []
for i in range(6):
    year = dff[i].strftime('%Y')
    mon = dff[i].strftime('%-m')
    day = dff[i].strftime('%-d')
    hour = dff[i].strftime('%-H')
    obs_utc.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 'atmp':(obs[i].atmp-32.)*5./9.}).astype(float))

In [5]:
dff = [pd.date_range(start='1/1/2001 6:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='5/2/2003 6:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='4/6/2006 6:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='1/1/2001 6:00', end='12/31/2018 4:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='1/1/2001 6:00', end='12/31/2018 4:00', freq='H').tz_localize(pytz.utc),
        pd.date_range(start='9/19/2007 23:00', end='12/31/2018 23:00', freq='H').tz_localize(pytz.utc)]
era_utc = []
for i in range(6):
    year = dff[i].strftime('%Y')
    mon = dff[i].strftime('%-m')
    day = dff[i].strftime('%-d')
    hour = dff[i].strftime('%-H')
    era_utc.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 'atmp':(era[i].atmp-32.)*5./9.}).astype(float))

In [6]:
# # obs_utc[0].loc[np.loobs_utc[0]['YEAR']==2010.]
# # obs_utc_tmin[0].loc[obs_utc_tmin[0]['YEAR']==2010.][60:]
# i = 0
# t_daymin_diff_pr = [pri_tmin[i]-obs_utc_tmin[i]['atmp'].values for i in range(6)]
# np.nanargmax(abs(t_daymin_diff_pr[0]))
# t_daymin_diff_pr[0][2302], pri_tmin[i][2302], obs_utc_tmin[i]['atmp'].values[2302]
# obs_utc_tmin[i][2302:]

In [7]:
dfd = [pd.date_range(start='1/2/2001', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='5/3/2003', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='4/7/2006', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='9/21/2007', end='12/31/2018', freq='D').tz_localize(pytz.utc)]
obs_utc_tmin = []
obs_utc_tmax = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    loc12 = obs_utc[i][obs_utc[i]['HOUR']==12.].index
    obs_utc_tmin.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':[np.nanmin(obs_utc[i].loc[loc12[j]:loc12[j+1]]['atmp']) for j in range(len(loc12)-1)]}).astype(float))
    obs_utc_tmax.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':[np.nanmax(obs_utc[i].loc[loc12[j]:loc12[j+1]]['atmp']) for j in range(len(loc12)-1)]}).astype(float))

<ipython-input-7-c784510ea627>:16: RuntimeWarning: All-NaN axis encountered
  'atmp':[np.nanmin(obs_utc[i].loc[loc12[j]:loc12[j+1]]['atmp']) for j in range(len(loc12)-1)]}).astype(float))
<ipython-input-7-c784510ea627>:18: RuntimeWarning: All-NaN axis encountered
  'atmp':[np.nanmax(obs_utc[i].loc[loc12[j]:loc12[j+1]]['atmp']) for j in range(len(loc12)-1)]}).astype(float))


In [8]:
dfd = [pd.date_range(start='1/2/2001', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='5/3/2003', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='4/7/2006', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='9/21/2007', end='12/31/2018', freq='D').tz_localize(pytz.utc)]
era_utc_tmin = []
era_utc_tmax = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    loc12 = era_utc[i][era_utc[i]['HOUR']==12.].index
    era_utc_tmin.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':[np.nanmin(era_utc[i].loc[loc12[j]:loc12[j+1]]['atmp']) for j in range(len(loc12)-1)]}).astype(float))
    era_utc_tmax.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':[np.nanmax(era_utc[i].loc[loc12[j]:loc12[j+1]]['atmp']) for j in range(len(loc12)-1)]}).astype(float))

In [88]:
prfna = ['East_Lansing_PRISM_tmin_stable_4km_19810101_20190101_42.6700_-84.4900.csv',\
        'East_Leland_PRISM_tmin_stable_4km_19810101_20190101_45.0300_-85.6700.csv',\
        'South_Haven_PRISM_tmin_stable_4km_19810101_20190101_42.3600_-86.2900.csv',\
        'Grand_Rapids_PRISM_tmin_stable_4km_19810101_20190101_42.8810_-85.5230.csv',\
        'Traverse_City_PRISM_tmin_stable_4km_19810101_20190101_44.7420_-85.5820.csv',\
        'Gaylord_PRISM_tmin_stable_4km_19810101_20190101_44.9100_-84.7200.csv']

locl = [-6574, -5723, -4653, -6574, -6574, -4121]
locr = [-1, -1, -1, -2, -2, -1]
pri_tmin = []
for i in range(6):
    df = pd.read_csv(f'./prism/stations/{prfna[i]}', header=[i for i in range(11)])
    pri_tmin.append((np.array(df[locl[i]:locr[i]])[:, 0] - 32.)*5./9.)
#     print(df[locl[i]:locr[i]])

In [89]:
prfna = ['East_Lansing_PRISM_tmax_stable_4km_20010101_20180101_42.6700_-84.4900.csv',\
        'East_Leland_PRISM_tmax_stable_4km_20010101_20180101_45.0300_-85.6700.csv',\
        'South_Haven_PRISM_tmax_stable_4km_20010101_20180101_42.3600_-86.2900.csv',\
        'Grand_Rapids_PRISM_tmax_stable_4km_20010101_20180101_42.8810_-85.5230.csv',\
        'Traverse_City_PRISM_tmax_stable_4km_20010101_20180101_44.7420_-85.5820.csv',\
        'Gaylord_PRISM_tmax_stable_4km_20010101_20180101_44.9100_-84.7200.csv']

locl = [1, 852, 1922, 1, 1, 2454]
locr = [6574, 6574, 6574, 6573, 6573, 6574]
pri_tmax = []
for i in range(6):
    df = pd.read_csv(f'./prism/stations/{prfna[i]}', header=[i for i in range(11)])
    pri_tmax.append((np.array(df[locl[i]:locr[i]])[:, 0] - 32.)*5./9.)
#     print(df[locl[i]:locr[i]])

In [87]:
# rmse_pritmin = [rmse(obs_utc_tmin[i]['atmp'], pd.DataFrame(pri_tmin[i])) for i in range(6)]
# rmse_pritmax = [rmse(obs_utc_tmax[i]['atmp'], pd.DataFrame(pri_tmax[i])) for i in range(6)]

# rmse_eratmin = [rmse(obs_utc_tmin[i]['atmp'], era_utc_tmin[i]['atmp']) for i in range(6)]
# rmse_eratmax = [rmse(obs_utc_tmax[i]['atmp'], era_utc_tmax[i]['atmp']) for i in range(6)]

In [79]:
locn = ['East Lansing', 'East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord']
df_rmse = pd.DataFrame({'Station':[locn[i] for i in range(6)],
                        'Number':[len(obs_utc_tmin[i]) for i in range(6)],
                        'DminT RMSE (C)':[round(rmse_pritmin[i],2) for i in range(6)],
                        'DmaxT RMSE (C)':[round(rmse_pritmax[i],2) for i in range(6)],
                        'DminT Bias (C)':[round(np.nanmean(pri_tmin[i]-obs_utc_tmin[i]['atmp'].values),2) for i in range(6)],
                        'DmaxT Bias (C)':[round(np.nanmean(pri_tmax[i]-obs_utc_tmax[i]['atmp'].values),2) for i in range(6)]})
df_rmse.loc['median'] = df_rmse.median()
df_rmse

,Station,Number,DminT RMSE (C),DmaxT RMSE (C),DminT Bias (C),DmaxT Bias (C)
0,East Lansing,6573.0,1.490,0.91,0.65,0.500
1,East Leland,5722.0,2.980,0.96,2.13,0.080
2,South Haven,4652.0,2.840,1.27,1.51,-0.340
3,Grand Rapids,6572.0,0.730,0.33,-0.01,-0.010
4,Traverse City,6572.0,1.030,0.58,0.29,-0.040
5,Gaylord,4120.0,1.620,0.83,-0.74,0.600
median,NaN,6147.0,1.555,0.87,0.47,0.035


In [19]:
locn = ['East Lansing', 'East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord']
df_rmse = pd.DataFrame({'Station':[locn[i] for i in range(6)],
                        'Number':[len(obs_utc_tmin[i]) for i in range(6)],
                        'DminT RMSE (C)':[round(rmse_eratmin[i],2) for i in range(6)],
                        'DmaxT RMSE (C)':[round(rmse_eratmax[i],2) for i in range(6)],
                        'DminT Bias (C)':[round(np.nanmean(era_utc_tmin[i]['atmp'].values-obs_utc_tmin[i]['atmp'].values),2) for i in range(6)],
                        'DmaxT Bias (C)':[round(np.nanmean(era_utc_tmax[i]['atmp'].values-obs_utc_tmax[i]['atmp'].values),2) for i in range(6)]})
df_rmse.loc['median'] = df_rmse.median()
df_rmse

,Station,Number,DminT RMSE (C),DmaxT RMSE (C),DminT Bias (C),DmaxT Bias (C)
0,East Lansing,6573.0,2.490,1.140,1.230,-0.100
1,East Leland,5722.0,4.270,3.150,2.760,-1.820
2,South Haven,4652.0,4.490,2.980,2.820,-1.830
3,Grand Rapids,6572.0,1.880,1.330,0.050,-0.600
4,Traverse City,6572.0,2.520,1.980,0.080,-1.230
5,Gaylord,4120.0,2.120,1.270,-0.050,0.340
median,NaN,6147.0,2.505,1.655,0.655,-0.915


In [52]:
# i = 0
# oobs = obs_utc_tmin[i][3652-307:3652-246]
# pd.DataFrame({'YEAR':oobs.YEAR, 'MONTH':oobs.MONTH, 'DAY':oobs.DAY, 
#             'OBS_tmin':oobs['atmp'].values, 'ERA_tmin':era_utc_tmin[i][3652-307:3652-246]['atmp'].values, 
#               'PRISM_tmin':pri_tmin[i][3652-307:3652-246],
#               'OBS_tmax':obs_utc_tmax[i][3652-307:3652-246]['atmp'].values,
#              'ERA_tmax':era_utc_tmax[i][3652-307:3652-246]['atmp'].values,
#              'PRISM_tmax':pri_tmax[i][3652-307:3652-246]}).to_csv('./TableTwoMon.csv')

In [53]:
# Daily time series
t_daymin_diff_pr = [pri_tmin[i]-obs_utc_tmin[i]['atmp'].values for i in range(6)]

plt.subplots(3,1,figsize=(20,15))
plt.subplot(3,1,1)
plt.plot(list(range(1,366)), t_daymin_diff_pr[0][3652-365:3652], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[1][3652-365-851:3652-851], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[2][3652-365-1921:3652-1921], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[3][3652-365:3652], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[4][3652-365:3652], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[5][835:835+365], 'y-o', lw=1.5, ms=4)
plt.axhline(color='k', lw=1, ls='--')
plt.ylim(-20, 20)
plt.xticks(np.arange(1, 366, step=30), fontsize = 12, fontweight='bold')
plt.ylabel('Temperature (\u2103) ', fontsize = 18, fontweight='bold')
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 13)
plt.title('PRISM-OBS: Daily minimum temperature (\u2103) in 2010', fontsize = 20, fontweight='bold')

plt.subplot(3,1,2)
plt.plot(list(range(1,366)), t_daymin_diff_pr[0][3652+365:3652+365*2], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[1][3652+365-851:3652+365*2-851], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[2][3652-365+365*2-1921:3652-365+365*3-1921], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[3][3652+365:3652+365*2], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[4][3652+365:3652+365*2], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[5][835+365*2:835+365*3], 'y-o', lw=1.5, ms=4)
plt.axhline(color='k', lw=1, ls='--')
plt.ylim(-20, 20)
plt.xticks(np.arange(1, 366, step=30), fontsize = 12, fontweight='bold')
plt.ylabel('Temperature (\u2103) ', fontsize = 18, fontweight='bold')
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 13)
plt.title('PRISM-OBS: Daily minimum temperature (\u2103) in 2012', fontsize = 20, fontweight='bold')

plt.subplot(3,1,3)
plt.plot(list(range(1,366)), t_daymin_diff_pr[0][3652+365*3:3652+365*4], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[1][3652+365*3-851:3652+365*4-851], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[2][3652-365+365*4-1921:3652-365+365*5-1921], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[3][3652+365*3:3652+365*4], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[4][3652+365*3:3652+365*4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymin_diff_pr[5][835+365*4:835+365*5], 'y-o', lw=1.5, ms=4)
plt.axhline(color='k', lw=1, ls='--')
plt.ylim(-20, 20)
plt.xticks(np.arange(1, 366, step=30), fontsize = 12, fontweight='bold')
plt.ylabel('Temperature (\u2103) ', fontsize = 18, fontweight='bold')
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 13)
plt.title('PRISM-OBS: Daily minimum temperature (\u2103) in 2014', fontsize = 20, fontweight='bold')
plt.savefig('./plot/thesis/PRISM_1yr_mindiff.png', bbox_inches='tight')
plt.close()

In [25]:
# Daily time series
t_daymax_diff_pr = [era_utc_tmax[i]['atmp'].values-obs_utc_tmax[i]['atmp'].values for i in range(6)]

plt.subplots(3,1,figsize=(20,15))
plt.subplot(3,1,1)
plt.plot(list(range(1,366)), t_daymax_diff_pr[0][3652-365:3652], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[1][3652-365-851:3652-851], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[2][3652-365-1921:3652-1921], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[3][3652-365:3652], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[4][3652-365:3652], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[5][835:835+365], 'y-o', lw=1.5, ms=4)
plt.axhline(color='k', lw=1, ls='--')
plt.ylim(-20, 20)
plt.xticks(np.arange(1, 366, step=30), fontsize = 12, fontweight='bold')
plt.ylabel('Temperature (\u2103) ', fontsize = 18, fontweight='bold')
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 13)
plt.title('ERA5-OBS: Daily maximum temperature (\u2103) in 2010', fontsize = 20, fontweight='bold')

plt.subplot(3,1,2)
plt.plot(list(range(1,366)), t_daymax_diff_pr[0][3652+365:3652+365*2], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[1][3652+365-851:3652+365*2-851], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[2][3652-365+365*2-1921:3652-365+365*3-1921], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[3][3652+365:3652+365*2], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[4][3652+365:3652+365*2], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[5][835+365*2:835+365*3], 'y-o', lw=1.5, ms=4)
plt.axhline(color='k', lw=1, ls='--')
plt.ylim(-20, 20)
plt.xticks(np.arange(1, 366, step=30), fontsize = 12, fontweight='bold')
plt.ylabel('Temperature (\u2103) ', fontsize = 18, fontweight='bold')
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 13)
plt.title('ERA5-OBS: Daily maximum temperature (\u2103) in 2012', fontsize = 20, fontweight='bold')

plt.subplot(3,1,3)
plt.plot(list(range(1,366)), t_daymax_diff_pr[0][3652+365*3:3652+365*4], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[1][3652+365*3-851:3652+365*4-851], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[2][3652-365+365*4-1921:3652-365+365*5-1921], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[3][3652+365*3:3652+365*4], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[4][3652+365*3:3652+365*4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(1,366)), t_daymax_diff_pr[5][835+365*4:835+365*5], 'y-o', lw=1.5, ms=4)
plt.axhline(color='k', lw=1, ls='--')
plt.ylim(-20, 20)
plt.xticks(np.arange(1, 366, step=30), fontsize = 12, fontweight='bold')
plt.ylabel('Temperature (\u2103) ', fontsize = 18, fontweight='bold')
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 13)
plt.title('ERA5-OBS: Daily maximum temperature (\u2103) in 2014', fontsize = 20, fontweight='bold')
plt.savefig('./plot/thesis/EAR5_1yr_maxdiff.png', bbox_inches='tight')
plt.close()

### Monthly time series

In [64]:
t_daymin_diff_pr = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    t_daymin_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':(pri_tmin[i]+pri_tmax[i])/2. - (obs_utc_tmin[i]['atmp'].values+obs_utc_tmax[i]['atmp'].values)/2.}).astype(float))
t_daymin_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymin_diff_pr]

plt.figure(figsize=(15,6))
plt.plot(list(range(1,217)), t_daymin_diff_pr[0], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(30,218)), t_daymin_diff_pr[1], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(65,218)), t_daymin_diff_pr[2], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[3], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(81,217)), t_daymin_diff_pr[5], 'y-o', lw=1.5, ms=4)

plt.ylim(-6, 6)
plt.axhline(color='k', lw=1, ls='--')
plt.xticks(np.arange(1, 218, step=24),['{0} Jan'.format(2*i+2001) for i in range(10)], fontweight='bold')
plt.xlabel('Time', fontweight='bold', fontsize=13)
plt.ylabel('Temperature (\u2103) ', fontweight='bold', fontsize=13)
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 12, frameon=False)
plt.title('PRISM-OBS: Monthly Means of Daily mean temperature(\u2103)', fontweight='bold', fontsize=15)
plt.savefig('./plot/thesis/PRI_meandiff.png', bbox_inches='tight')
plt.close()

In [28]:
# t_daymin_diff_pr = []
# for i in range(6):
#     year = dfd[i].strftime('%Y')
#     mon = dfd[i].strftime('%-m')
#     day = dfd[i].strftime('%-d')
#     hour = dfd[i].strftime('%-H')
#     t_daymin_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
#                                       'atmp':(pri_tmin[i]+pri_tmax[i])/2. - (obs_utc_tmin[i]['atmp'].values+obs_utc_tmax[i]['atmp'].values)/2.}).astype(float))
# t_daymin_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymin_diff_pr]
# np.argmax(np.array(t_daymin_diff_pr[2]))
# 2009/9 2007/11

# t_daymin_diff_pr = []
# for i in range(6):
#     year = dfd[i].strftime('%Y')
#     mon = dfd[i].strftime('%-m')
#     day = dfd[i].strftime('%-d')
#     hour = dfd[i].strftime('%-H')
#     t_daymin_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
#                                       'atmp':(era_utc_tmin[i]['atmp'].values+era_utc_tmax[i]['atmp'].values)/2. - (obs_utc_tmin[i]['atmp'].values+obs_utc_tmax[i]['atmp'].values)/2.}).astype(float))
# t_daymin_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymin_diff_pr]
# np.argmax(np.array(t_daymin_diff_pr[2]))
# 2007/11 2009/9
# t_daymin_diff_pr[2][30:42]

In [16]:
t_daymin_diff_pr = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    t_daymin_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':(era_utc_tmin[i]['atmp'].values+era_utc_tmax[i]['atmp'].values)/2. - (obs_utc_tmin[i]['atmp'].values+obs_utc_tmax[i]['atmp'].values)/2.}).astype(float))
t_daymin_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymin_diff_pr]

plt.figure(figsize=(15,6))
# [el_loc,gl_loc,ele_loc,sh_loc,gr_loc,tvc_loc]
plt.plot(list(range(1,217)), t_daymin_diff_pr[0], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(30,218)), t_daymin_diff_pr[1], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(65,218)), t_daymin_diff_pr[2], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[3], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(81,217)), t_daymin_diff_pr[5], 'y-o', lw=1.5, ms=4)

plt.ylim(-6, 6)
plt.axhline(color='k', lw=1, ls='--')
plt.xticks(np.arange(1, 218, step=24),['{0} Jan'.format(2*i+2001) for i in range(10)], fontweight='bold')
plt.xlabel('Time', fontweight='bold', fontsize=13)
plt.ylabel('Temperature (\u2103) ', fontweight='bold', fontsize=13)
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 12, frameon=False)
plt.title('ERA5-OBS: Monthly Means of Daily mean temperature(\u2103)', fontweight='bold', fontsize=15)
plt.savefig('./plot/thesis/ERA_meandiff.png', bbox_inches='tight')
plt.close()

In [72]:
t_daymin_diff_pr = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    t_daymin_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':(pri_tmin[i] - obs_utc_tmin[i]['atmp'].values)}).astype(float))
t_daymin_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymin_diff_pr]


plt.figure(figsize=(15,6))
# [el_loc,gl_loc,ele_loc,sh_loc,gr_loc,tvc_loc]
plt.plot(list(range(1,217)), t_daymin_diff_pr[0], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(30,218)), t_daymin_diff_pr[1], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(65,218)), t_daymin_diff_pr[2], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[3], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(81,217)), t_daymin_diff_pr[5], 'y-o', lw=1.5, ms=4)

plt.ylim(-11, 11)
plt.axhline(color='k', lw=1, ls='--')
plt.xticks(np.arange(1, 218, step=24),['{0} Jan'.format(2*i+2001) for i in range(10)], fontweight='bold')
plt.xlabel('Time', fontweight='bold', fontsize=13)
plt.ylabel('Temperature (\u2103) ', fontweight='bold', fontsize=13)
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 12, frameon=False)
plt.title('PRISM-OBS: Daily min temperature monthly mean (\u2103)', fontweight='bold', fontsize=15)
plt.savefig('./plot/thesis/PRI_mindiff.png', bbox_inches='tight')
plt.close()

In [30]:
t_daymin_diff_pr = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    t_daymin_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':(era_utc_tmin[i]['atmp'].values - obs_utc_tmin[i]['atmp'].values)}).astype(float))
t_daymin_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymin_diff_pr]

plt.figure(figsize=(15,6))
# [el_loc,gl_loc,ele_loc,sh_loc,gr_loc,tvc_loc]
plt.plot(list(range(1,217)), t_daymin_diff_pr[0], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(30,218)), t_daymin_diff_pr[1], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(65,218)), t_daymin_diff_pr[2], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[3], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymin_diff_pr[4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(81,217)), t_daymin_diff_pr[5], 'y-o', lw=1.5, ms=4)

plt.ylim(-11, 11)
plt.axhline(color='k', lw=1, ls='--')
plt.xticks(np.arange(1, 218, step=24),['{0} Jan'.format(2*i+2001) for i in range(10)], fontweight='bold')
plt.xlabel('Time', fontweight='bold', fontsize=13)
plt.ylabel('Temperature (\u2103) ', fontweight='bold', fontsize=13)
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 12, frameon=False)
plt.title('ERA5-OBS: Daily min temperature monthly mean (\u2103)', fontweight='bold', fontsize=15)
plt.savefig('./plot/thesis/ERA_mindiff.png', bbox_inches='tight')
plt.close()

In [73]:
t_daymax_diff_pr = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    t_daymax_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':(pri_tmax[i] - obs_utc_tmax[i]['atmp'].values)}).astype(float))
t_daymax_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymax_diff_pr]

plt.figure(figsize=(15,6))
# [el_loc,gl_loc,ele_loc,sh_loc,gr_loc,tvc_loc]
plt.plot(list(range(1,217)), t_daymax_diff_pr[0], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(30,218)), t_daymax_diff_pr[1], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(65,218)), t_daymax_diff_pr[2], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymax_diff_pr[3], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymax_diff_pr[4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(81,217)), t_daymax_diff_pr[5], 'y-o', lw=1.5, ms=4)

plt.ylim(-8, 8)
plt.axhline(color='k', lw=1, ls='--')
plt.xticks(np.arange(1, 218, step=24),['{0} Jan'.format(2*i+2001) for i in range(10)], fontweight='bold')
plt.xlabel('Time', fontweight='bold', fontsize=13)
plt.ylabel('Temperature (\u2103) ', fontweight='bold', fontsize=13)
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 12, frameon=False)
plt.title('PRISM-OBS: Daily max temperature monthly mean (\u2103)', fontweight='bold', fontsize=15)
plt.savefig('./plot/thesis/PRI_maxdiff.png', bbox_inches='tight')
plt.close()

In [31]:
t_daymax_diff_pr = []
for i in range(6):
    year = dfd[i].strftime('%Y')
    mon = dfd[i].strftime('%-m')
    day = dfd[i].strftime('%-d')
    hour = dfd[i].strftime('%-H')
    t_daymax_diff_pr.append(pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day, 'HOUR':hour, 
                                      'atmp':(era_utc_tmax[i]['atmp'].values - obs_utc_tmax[i]['atmp'].values)}).astype(float))
t_daymax_diff_pr = [diff.groupby([diff.YEAR,diff.MONTH])['atmp'].mean() for diff in t_daymax_diff_pr]

plt.figure(figsize=(15,6))
# [el_loc,gl_loc,ele_loc,sh_loc,gr_loc,tvc_loc]
plt.plot(list(range(1,217)), t_daymax_diff_pr[0], 'r-o', lw=1.5, ms=4)
plt.plot(list(range(30,218)), t_daymax_diff_pr[1], 'b-o', lw=1.5, ms=4)
plt.plot(list(range(65,218)), t_daymax_diff_pr[2], 'g-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymax_diff_pr[3], 'c-o', lw=1.5, ms=4)
plt.plot(list(range(1,217)), t_daymax_diff_pr[4], 'm-o', lw=1.5, ms=4)
plt.plot(list(range(81,217)), t_daymax_diff_pr[5], 'y-o', lw=1.5, ms=4)

plt.ylim(-8, 8)
plt.axhline(color='k', lw=1, ls='--')
plt.xticks(np.arange(1, 218, step=24),['{0} Jan'.format(2*i+2001) for i in range(10)], fontweight='bold')
plt.xlabel('Time', fontweight='bold', fontsize=13)
plt.ylabel('Temperature (\u2103) ', fontweight='bold', fontsize=13)
plt.legend(['East Lansing','East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord'],fontsize = 12, frameon=False)
plt.title('ERA5-OBS: Daily max temperature monthly mean (\u2103)', fontweight='bold', fontsize=15)
plt.savefig('./plot/thesis/ERA_maxdiff.png', bbox_inches='tight')
plt.close()

### GDD

In [32]:
from math import atan, sqrt, cos

In [33]:
def compute_gddC(tmax, tmin):
    Tmax = tmax
    Tmin = tmin
    
    if Tmax < Tmin:
        Tmax = Tmin + .05
    
    tmean = (Tmax + Tmin) / 2.
    tbase = 4.
    
    if Tmax < tbase:
        gdd = 0.
    elif Tmin > tbase:
        gdd = tmean - tbase
    else:
        w = (Tmax - Tmin) / 2.
        y = (tbase - tmean) / w
        try:
            a = atan(y / sqrt(-y*y + 1.))
            gdd = ((w * cos(a) - (tbase - tmean) * (3.1415927 / 2. - a)) / 3.1415927)
        except ZeroDivisionError:
            gdd = np.nan
    
    return gdd

In [34]:
# obs_gdd = []
obs_utc_gdd = []
era_gdd = []
pri_gdd = []
for si in range(6):
#     obs_gdd.append(np.array(list(map(compute_gdd, obs_tmax[si], obs_tmin[si]))))
    obs_utc_gdd.append(np.array(list(map(compute_gddC, obs_utc_tmax[si]['atmp'].values, obs_utc_tmin[si]['atmp'].values))))
    era_gdd.append(np.array(list(map(compute_gddC, era_utc_tmax[si]['atmp'].values, era_utc_tmin[si]['atmp'].values))))
    pri_gdd.append(np.array(list(map(compute_gddC, pri_tmax[si], pri_tmin[si]))))

<ipython-input-33-dd3673a84bb4>:19: RuntimeWarning: divide by zero encountered in double_scalars
  a = atan(y / sqrt(-y*y + 1.))


In [35]:
# obs_gdds = [pd.Series(obs_gdd[i], index=obs_tmax[i].index) for i in range(6)]
# era_gdds = [pd.Series(era_gdd[i], index=obs_tmax[i].index) for i in range(6)]
obs_utc_gdds = [pd.Series(obs_utc_gdd[i], index=obs_utc_tmax[i].index) for i in range(6)]
era_gdds = [pd.Series(era_gdd[i], index=obs_utc_tmax[i].index) for i in range(6)]
pri_gdds = [pd.Series(pri_gdd[i], index=obs_utc_tmax[i].index) for i in range(6)]

In [36]:
rmse_eragdd = [rmse(obs_utc_gdds[i], era_gdds[i]) for i in range(6)]
rmse_prigdd = [rmse(obs_utc_gdds[i], pri_gdds[i]) for i in range(6)]

In [38]:
locn = ['East Lansing', 'East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord']
df_rmse = pd.DataFrame({'Station':[locn[i] for i in range(6)],
                        'Number':[len(obs_utc_tmax[i]) for i in range(6)],
                        'GDD RMSE (ERA5)':[round(rmse_eragdd[i],2) for i in range(6)],
                        'GDD RMSE (PRISM)':[round(rmse_prigdd[i],2) for i in range(6)],
                        'GDD Bias (ERA5)':[np.round(np.nanmean(era_gdds[i].values-obs_utc_gdds[i].values),2) for i in range(6)],
                        'GDD Bias (PRISM)':[np.round(np.nanmean(pri_gdds[i].values-obs_utc_gdds[i].values),2) for i in range(6)]})
df_rmse.loc['median'] = df_rmse.median()
df_rmse

,Station,Number,GDD RMSE (ERA5),GDD RMSE (PRISM),GDD Bias (ERA5),GDD Bias (PRISM)
0,East Lansing,6573.0,0.99,0.77,0.490,0.380
1,East Leland,5722.0,1.79,1.07,0.200,0.590
2,South Haven,4652.0,1.96,1.00,0.270,0.360
3,Grand Rapids,6572.0,0.71,0.34,-0.070,0.000
4,Traverse City,6572.0,0.99,0.47,-0.130,0.050
5,Gaylord,4120.0,0.80,0.49,0.300,0.020
median,NaN,6147.0,0.99,0.63,0.235,0.205


### Scatter plot

In [34]:
locn = ['East Lansing', 'East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord']
plt.subplots(6, 4, figsize=(12, 16))
for si in range(6):
    ax1 = plt.subplot(6, 4, si*4+1)
    plt.scatter(obs_utc_tmin[si]['atmp'].values, era_utc_tmin[si]['atmp'].values, s=1, c='k')
    plt.xlim(-40, 40)
    plt.ylim(-40, 40)
    plt.text(.05, .9, locn[si], fontsize='large', fontweight='bold', c='k', ha='left', va='center', transform=ax1.transAxes)
    plt.plot(ax1.get_xlim(), ax1.get_ylim(), 'r')
    plt.ylabel('GRID', fontsize='large', fontweight='bold')
    if si == 5:
        plt.xlabel('OBS', fontsize='large', fontweight='bold')
    if si == 0:
        plt.title('ERA5 TMIN', fontsize='large', fontweight='bold')
    ax2 = plt.subplot(6, 4, si*4+2)
    plt.scatter(obs_utc_tmax[si]['atmp'].values, era_utc_tmax[si]['atmp'].values, s=1, c='k')
    plt.xlim(-40, 40)
    plt.ylim(-40, 40)
    plt.plot(ax2.get_xlim(), ax2.get_ylim(), 'r')
    if si == 5:
        plt.xlabel('OBS', fontsize='large', fontweight='bold')
    if si == 0:
        plt.title('ERA5 TMAX', fontsize='large', fontweight='bold')
    ax3 = plt.subplot(6, 4, si*4+3)
    plt.scatter(obs_utc_tmin[si]['atmp'].values, pri_tmin[si], s=1, c='k')
    plt.xlim(-40, 40)
    plt.ylim(-40, 40)
    plt.plot(ax3.get_xlim(), ax3.get_ylim(), 'r')
    if si == 5:
        plt.xlabel('OBS', fontsize='large', fontweight='bold')
    if si == 0:
        plt.title('PRISM TMIN', fontsize='large', fontweight='bold')
    ax4 = plt.subplot(6, 4, si*4+4)
    plt.scatter(obs_utc_tmax[si]['atmp'].values, pri_tmax[si], s=1, c='k')
    plt.xlim(-40, 40)
    plt.ylim(-40, 40)
    plt.plot(ax4.get_xlim(), ax4.get_ylim(), 'r')
    if si == 5:
        plt.xlabel('OBS', fontsize='large', fontweight='bold')
    if si == 0:
        plt.title('PRISM TMAX', fontsize='large', fontweight='bold')
plt.savefig('./plot/thesis/DailyTempscatter.png', bbox_inches='tight')
plt.close()

### Daily Min Temp hit rate

In [49]:
t_daymin_obs = [np.array(obs_utc_tmin[i]['atmp'].values) for i in range(6)]
# t_daymin_era = [np.array(era_utc_tmin[i]['atmp'].values) for i in range(6)]
t_daymin_era = pri_tmin
l = [len(DminT) for DminT in t_daymin_obs]
ratioc = [np.zeros((ll, 5))*np.nan for ll in l]
for i in range(6):
    for j in range(l[i]):
        if np.logical_and((t_daymin_obs[i][j] <= 0),(t_daymin_obs[i][j] > -5)):
            t_daymin_obs[i][j] = 1
        elif np.logical_and((t_daymin_obs[i][j] <= -5),(t_daymin_obs[i][j] > -10)):
            t_daymin_obs[i][j] = 2
        elif np.logical_and((t_daymin_obs[i][j] <= -10),(t_daymin_obs[i][j] > -15)):
            t_daymin_obs[i][j] = 3
        elif np.logical_and((t_daymin_obs[i][j] <= -15),(t_daymin_obs[i][j] > -20)):
            t_daymin_obs[i][j] = 4
        elif np.logical_and((t_daymin_obs[i][j] <= -20),(t_daymin_obs[i][j] > -25)):
            t_daymin_obs[i][j] = 5
        else:
            t_daymin_obs[i][j] = np.nan
            
        if np.logical_and((t_daymin_era[i][j] <= 0),(t_daymin_era[i][j] > -5)):
            t_daymin_era[i][j] = 1
        elif np.logical_and((t_daymin_era[i][j] <= -5),(t_daymin_era[i][j] > -10)):
            t_daymin_era[i][j] = 2
        elif np.logical_and((t_daymin_era[i][j] <= -10),(t_daymin_era[i][j] > -15)):
            t_daymin_era[i][j] = 3
        elif np.logical_and((t_daymin_era[i][j] <= -15),(t_daymin_era[i][j] > -20)):
            t_daymin_era[i][j] = 4
        elif np.logical_and((t_daymin_era[i][j] <= -20),(t_daymin_era[i][j] > -25)):
            t_daymin_era[i][j] = 5
        else:
            t_daymin_era[i][j] = np.nan

for i in range(6):
    for j in range(5):
        ratioc[i][:,j] = np.where(np.logical_and((t_daymin_obs[i][:] == j+1),(t_daymin_era[i][:] == j+1)), 1.0, ratioc[i][:,j])
        ratioc[i][:,j] = np.where(np.logical_and((t_daymin_obs[i][:] == j+1),(t_daymin_era[i][:] != j+1)), 0.0, ratioc[i][:,j])

In [50]:
dfd = [pd.date_range(start='1/2/2001', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='5/3/2003', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='4/7/2006', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='9/21/2007', end='12/31/2018', freq='D').tz_localize(pytz.utc)]
si = 0
dff = dfd[si]
year = dff.strftime('%Y')
mon = dff.strftime('%-m')
day = dff.strftime('%-d')
ratiotc_era = pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day,
                         '-5~0':ratioc[si][:,0],
                         '-10~-5':ratioc[si][:,1],
                         '-15~-10':ratioc[si][:,2],
                         '-20~-15':ratioc[si][:,3],
                         '-25~-20':ratioc[si][:,4]}).astype(float)

In [14]:
# category = ['-5~0', '-10~-5', '-15~-10', '-20~-15', '-25~-20']
# rt = [ratiotc_el.groupby(ratiotc_el.MONTH)[category[ci]].sum() / ratiotc_el.groupby(ratiotc_el.MONTH)[category[ci]].count() for ci in range(5)]
# st = [[str(float(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)[category[ci]].sum())[i]))
#       +'/'+str(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)[category[ci]].count())[i]) for i in  range(12) ] for ci in range(5)]
# ratiotca_el = pd.DataFrame({
#                          '-5~0':ratiotc_el.groupby(ratiotc_el.MONTH)['-5~0'].sum() / ratiotc_el.groupby(ratiotc_el.MONTH)['-5~0'].count(),
#                          '-10~-5':ratiotc_el.groupby(ratiotc_el.MONTH)['-10~-5'].sum() / ratiotc_el.groupby(ratiotc_el.MONTH)['-10~-5'].count(),
#                          '-15~-10':ratiotc_el.groupby(ratiotc_el.MONTH)['-15~-10'].sum() / ratiotc_el.groupby(ratiotc_el.MONTH)['-15~-10'].count(),
#                          '-20~-15':ratiotc_el.groupby(ratiotc_el.MONTH)['-20~-15'].sum() / ratiotc_el.groupby(ratiotc_el.MONTH)['-20~-15'].count(),
#                          '-25~-20':ratiotc_el.groupby(ratiotc_el.MONTH)['-25~-20'].sum() / ratiotc_el.groupby(ratiotc_el.MONTH)['-25~-20'].count(),
#                          '-5~0_era/obs':[str(float(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-5~0'].sum())[i]))+'/'+str(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-5~0'].count())[i]) for i in  range(12) ],
#                          '-10~-5_era/obs':[str(float(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-10~-5'].sum())[i]))+'/'+str(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-10~-5'].count())[i]) for i in  range(12) ],
#                          '-15~-10_era/obs':[str(float(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-15~-10'].sum())[i]))+'/'+str(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-15~-10'].count())[i]) for i in  range(12) ],
#                          '-20~-15_era/obs':[str(float(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-20~-15'].sum())[i]))+'/'+str(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-20~-15'].count())[i]) for i in  range(12) ],
#                          '-25~-20_era/obs':[str(float(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-25~-20'].sum())[i]))+'/'+str(np.array(ratiotc_el.groupby(ratiotc_el.MONTH)['-25~-20'].count())[i]) for i in  range(12) ],
# })
# ratiotca_el

In [11]:
# category = ['-5~0', '-10~-5', '-15~-10', '-20~-15', '-25~-20']
# rt = [ratiotc_era.groupby(ratiotc_era.MONTH)[category[ci]].sum() / ratiotc_era.groupby(ratiotc_era.MONTH)[category[ci]].count() for ci in range(5)]
# st = [[f'{int(np.array(ratiotc_era.groupby(ratiotc_era.MONTH)[category[ci]].sum())[i])}'
#       +'/'+str(np.array(ratiotc_era.groupby(ratiotc_era.MONTH)[category[ci]].count())[i])
#       +'='+f'{rt[ci].values[i]:.2f}' for i in  range(12) ] for ci in range(5)]
# table = pd.DataFrame({category[ci]:st[ci] for ci in range(5)}, index=range(1,13))
# table

In [90]:
t_daymin_obs = [np.array(obs_utc_tmin[i]['atmp'].values) for i in range(6)]
# t_daymin_era = [np.array(era_utc_tmin[i]['atmp'].values) for i in range(6)]
t_daymin_era = [tmin for tmin in pri_tmin]
l = [len(DminT) for DminT in t_daymin_obs]
ratioc = [np.zeros(ll)*np.nan for ll in l]
for i in range(6):
    for j in range(l[i]):
        if t_daymin_obs[i][j] <= 0:
            t_daymin_obs[i][j] = 1
        else:
            t_daymin_obs[i][j] = np.nan
            
        if t_daymin_era[i][j] <= 0:
            t_daymin_era[i][j] = 1
        else:
            t_daymin_era[i][j] = np.nan

for i in range(6):
    ratioc[i][:] = np.where(np.logical_and((t_daymin_obs[i][:] == 1),(t_daymin_era[i][:] == 1)), 1.0, ratioc[i][:])
    ratioc[i][:] = np.where(np.logical_and((t_daymin_obs[i][:] == 1),(t_daymin_era[i][:] != 1)), 0.0, ratioc[i][:])

In [101]:
dfd = [pd.date_range(start='1/2/2001', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='5/3/2003', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='4/7/2006', end='12/31/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='1/2/2001', end='12/30/2018', freq='D').tz_localize(pytz.utc),
        pd.date_range(start='9/21/2007', end='12/31/2018', freq='D').tz_localize(pytz.utc)]
si = 5
dff = dfd[si]
year = dff.strftime('%Y')
mon = dff.strftime('%-m')
day = dff.strftime('%-d')
ratiotc_era = pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day,
                         'HitRate':ratioc[si][:]}).astype(float)

In [103]:
locn = ['East Lansing', 'East Leland', 'South Haven', 'Grand Rapids', 'Traverse City', 'Gaylord']
rt = [ratiotc_era.groupby(ratiotc_era.MONTH)['HitRate'].sum() / ratiotc_era.groupby(ratiotc_era.MONTH)['HitRate'].count()]
st = [f'{int(np.array(ratiotc_era.groupby(ratiotc_era.MONTH)["HitRate"].sum())[i])}'
      +'/'+str(np.array(ratiotc_era.groupby(ratiotc_era.MONTH)["HitRate"].count())[i])
      +'='+f'{np.array(rt[0])[i]:.2f}' for i in  range(12)] 
table = pd.DataFrame({locn[si]:st}, index=range(1,13))


In [104]:
rt = []
for si in range(6):
    dff = dfd[si]
    year = dff.strftime('%Y')
    mon = dff.strftime('%-m')
    day = dff.strftime('%-d')
    ratiotc_era = pd.DataFrame({'YEAR':year, 'MONTH':mon, 'DAY':day,
                             'HitRate':ratioc[si][:]}).astype(float)
    rt.append(np.array(ratiotc_era.groupby(ratiotc_era.MONTH)['HitRate'].sum() / ratiotc_era.groupby(ratiotc_era.MONTH)['HitRate'].count()))


In [113]:
# np.round(np.nanmedian(np.array(rt), axis=1), 2)
np.nanmedian(np.array(rt)[:,0])

0.9859509511108272